Custom Built Residual Neural Network

In [1]:
# Setup
import tensorflow as tf
print("TensorFlow version", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input, BatchNormalization, Activation, Dropout, AveragePooling2D
from tensorflow.keras import Model, losses, models, layers, datasets, optimizers
import matplotlib.pyplot as plt


TensorFlow version 2.12.0


In [2]:
fashion_mnist = datasets.mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train/255, x_test/255

# Channels Dimms
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

# Batching and shuffling the dataset
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(512)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(512)

11490434/11490434 [==============================] - 1s 0us/step


In [3]:
def residual_block(X, num_filters: int, stride: int = 1, kernel_size: int = 3,
                   activation: str = "relu", bn: bool = True, conv_first: bool = True):

  conv_layer = Conv2D(num_filters,
                      kernel_size=kernel_size,
                      strides=stride,
                      padding="same",
                      kernel_regularizer=tf.keras.regularizers.L2(1e-4))

  if conv_first:
    X = conv_layer(X)
    if bn:
      X = BatchNormalization()(X)
    if activation is not None:
      X = Activation(activation)(X)
      X = Dropout(0.2)(X)
  else:
    if bn:
      X = BatchNormalization()(X)
    if activation is not None:
      X = Activation(activation)(X)
    X = conv_layer(X)

  return X

In [4]:
# depth should be 9n+2 (eg 56 or 110)
depth = 11

# Model definition
num_filters_in = 32
num_res_block = int((depth - 2) / 9)

inputs = Input(shape=(28,28,1))

# ResNet V2 performs Conv2D on X before spiting into two path
X = residual_block(X=inputs, num_filters=num_filters_in, conv_first=True)

# Building stack of residual units
for stage in range(3):
    for unit_res_block in range(num_res_block):
        activation = 'relu'
        bn = True
        stride = 1
        # First layer and first stage
        if stage == 0:
            num_filters_out = num_filters_in * 4
            if unit_res_block == 0:
                activation = None
                bn = False
            # First layer but not first stage
        else:
            num_filters_out = num_filters_in * 2
            if unit_res_block == 0:
                stride = 2

        # bottleneck residual unit
        y = residual_block(X,
                           num_filters=num_filters_in,
                           kernel_size=1,
                           stride=stride,
                           activation=activation,
                           bn=bn,
                           conv_first=False)
        y = residual_block(y,
                           num_filters=num_filters_in,
                           conv_first=False)
        y = residual_block(y,
                           num_filters=num_filters_out,
                           kernel_size=1,
                           conv_first=False)
        if unit_res_block == 0:
            # linear projection residual shortcut connection to match
            # changed dims
            X = residual_block(X=X,
                               num_filters=num_filters_out,
                               kernel_size=1,
                               stride=stride,
                               activation=None,
                               bn=False)
        X = tf.keras.layers.add([X, y])
    num_filters_in = num_filters_out

In [5]:
X = BatchNormalization()(X)
X = Activation('relu')(X)
X = AveragePooling2D(pool_size=1)(X)
y = Flatten()(X)
y = Dense(512, activation='relu')(y)
y = BatchNormalization()(y)
y = Dropout(0.5)(y)

outputs = Dense(10,
                activation='softmax')(y)

# Instantiate model.
model = Model(inputs=inputs, outputs=outputs)

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 28, 28, 32)   320         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 28, 28, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 28, 28, 32)   0           ['batch_normalization[0][0]']

In [8]:
model.compile(optimizer=optimizers.Adam(learning_rate=10), loss=losses.sparse_categorical_crossentropy, metrics=["accuracy"])
history = model.fit(x_train, y_train, batch_size=256, epochs=5)

Epoch 1/5
235/235 [==============================] - 1590s 7s/step - loss: 9629.3652 - accuracy: 0.7395
Epoch 2/5
235/235 [==============================] - 1579s 7s/step - loss: 15216.8770 - accuracy: 0.7736
Epoch 3/5
235/235 [==============================] - 1577s 7s/step - loss: 36899.5742 - accuracy: 0.8027
Epoch 4/5
235/235 [==============================] - 1578s 7s/step - loss: 32926.3945 - accuracy: 0.8729
Epoch 5/5
235/235 [==============================] - 1584s 7s/step - loss: 45294.4141 - accuracy: 0.8876


In [9]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 62s 195ms/step - loss: 134994.6562 - accuracy: 0.3330


[134994.65625, 0.3330000042915344]